1. Load the model Yeast8

In [28]:
from cobra.io import read_sbml_model
from cobra import Reaction, Metabolite

model = read_sbml_model('yeastGEM.xml')

2. Running model with standard Meidum 

In [29]:
# optimize for biomass growth with standard medium

In [30]:
model.optimize()                               
print(model.objective.value)

0.08374778664999834


In [31]:
#Create reaction dictionary

In [32]:
xyodict = {1: "r_2105", # xylitol transport
           2: "r_1719", # D-xylose transport
           3: "r_1093", # xylose reductase
           4: "r_1092"  # xylitol dehydrogenase
          }

In [33]:
#Check the standard medium. (medium that has as carbon source Glucose)


In [34]:
medium_glu = model.medium
for element in medium_glu.keys():
    print(model.reactions.get_by_id(element).name, model.reactions.get_by_id(element), medium_glu[element])

ammonium exchange r_1654: s_0420[e] <=>  1000.0
D-glucose exchange r_1714: s_0565[e] <=>  1.0
H+ exchange r_1832: s_0796[e] <=>  1000.0
iron(2+) exchange r_1861: s_0925[e] <=>  1000.0
oxygen exchange r_1992: s_1277[e] <=>  1000.0
phosphate exchange r_2005: s_1324[e] <=>  1000.0
potassium exchange r_2020: s_1374[e] <=>  1000.0
sodium exchange r_2049: s_1438[e] <=>  1000.0
sulphate exchange r_2060: s_1468[e] <=>  1000.0
water exchange r_2100: s_0805[e] <=>  1000.0
chloride exchange r_4593: s_4200[e] <=>  1000.0
Cu2(+) exchange r_4594: s_4201[e] <=>  1000.0
Mn(2+) exchange r_4595: s_4202[e] <=>  1000.0
Zn(2+) exchange r_4596: s_4203[e] <=>  1000.0
Mg(2+) exchange r_4597: s_4204[e] <=>  1000.0
Ca(2+) exchange r_4600: s_4199[e] <=>  1000.0


In [35]:
#Run the model with the standard medium 

In [36]:
solution = model.optimize()
print(model.summary())
for key in xyodict.keys():
    print(model.reactions.get_by_id(xyodict[key]).name, model.reactions.get_by_id(xyodict[key]).flux)

Objective
1.0 r_2111 = 0.08374778664999834

Uptake
------
Metabolite Reaction      Flux  C-Number   C-Flux
 s_0420[e]   r_1654    0.5988         0    0.00%
 s_0565[e]   r_1714         1         6  100.00%
 s_0796[e]   r_1832   0.04236         0    0.00%
 s_0925[e]   r_1861 2.546E-06         0    0.00%
 s_1277[e]   r_1992      2.25         0    0.00%
 s_1324[e]   r_2005    0.5449         0    0.00%
 s_1374[e]   r_2020  0.000304         0    0.00%
 s_1438[e]   r_2049 0.0003325         0    0.00%
 s_1468[e]   r_2060  0.007203         0    0.00%
 s_4200[e]   r_4593  0.000108         0    0.00%
 s_4201[e]   r_4594 5.519E-05         0    0.00%
 s_4202[e]   r_4595 0.0002286         0    0.00%
 s_4203[e]   r_4596 6.264E-05         0    0.00%
 s_4204[e]   r_4597 0.0001041         0    0.00%
 s_4199[e]   r_4600 1.817E-05         0    0.00%

Secretion
---------
Metabolite Reaction      Flux  C-Number   C-Flux
 s_0458[e]   r_1672     -2.39         1  100.00%
 s_0776[e]   r_1814 -5.41E-06         2

We can see how when the model is run with the standard medium (no xylose is present) the Xylitol transporter, Xylose transporter, Xylose Reductase (XR) and Xylitol Dehydrogenase (XDH) have flux equal to 0.  That means that with out Xylose in the media all the pathway is not active.

3. Medium Change, trying different proportions of Glucose and Xylose

3.1. Medium without glucose and with Xylose

In [37]:

medium_xyl = medium_glu
with model: 
    del medium_xyl['r_1714']
    medium_xyl['r_1718'] = 1.0

for element in medium_xyl.keys():
    print(model.reactions.get_by_id(element).name, model.reactions.get_by_id(element), medium_xyl[element])

ammonium exchange r_1654: s_0420[e] <=>  1000.0
H+ exchange r_1832: s_0796[e] <=>  1000.0
iron(2+) exchange r_1861: s_0925[e] <=>  1000.0
oxygen exchange r_1992: s_1277[e] <=>  1000.0
phosphate exchange r_2005: s_1324[e] <=>  1000.0
potassium exchange r_2020: s_1374[e] <=>  1000.0
sodium exchange r_2049: s_1438[e] <=>  1000.0
sulphate exchange r_2060: s_1468[e] <=>  1000.0
water exchange r_2100: s_0805[e] <=>  1000.0
chloride exchange r_4593: s_4200[e] <=>  1000.0
Cu2(+) exchange r_4594: s_4201[e] <=>  1000.0
Mn(2+) exchange r_4595: s_4202[e] <=>  1000.0
Zn(2+) exchange r_4596: s_4203[e] <=>  1000.0
Mg(2+) exchange r_4597: s_4204[e] <=>  1000.0
Ca(2+) exchange r_4600: s_4199[e] <=>  1000.0
D-xylose exchange r_1718: s_0579[e] -->  1.0


In [38]:
with model:
    model.medium = medium_xyl
    solution2 = model.optimize()
    print(model.summary())
    for key in xyodict.keys():
        print(model.reactions.get_by_id(xyodict[key]).name, model.reactions.get_by_id(xyodict[key]).flux)

Objective
1.0 r_2111 = 0.06735836314274217

Uptake
------
Metabolite Reaction      Flux  C-Number   C-Flux
 s_0420[e]   r_1654    0.4816         0    0.00%
 s_0579[e]   r_1718         1         5  100.00%
 s_0796[e]   r_1832   0.03407         0    0.00%
 s_0925[e]   r_1861 2.048E-06         0    0.00%
 s_1277[e]   r_1992     1.984         0    0.00%
 s_1324[e]   r_2005    0.4383         0    0.00%
 s_1374[e]   r_2020 0.0002445         0    0.00%
 s_1438[e]   r_2049 0.0002674         0    0.00%
 s_1468[e]   r_2060  0.005794         0    0.00%
 s_4200[e]   r_4593 8.689E-05         0    0.00%
 s_4201[e]   r_4594 4.439E-05         0    0.00%
 s_4202[e]   r_4595 0.0001839         0    0.00%
 s_4203[e]   r_4596 5.038E-05         0    0.00%
 s_4204[e]   r_4597  8.37E-05         0    0.00%
 s_4199[e]   r_4600 1.462E-05         0    0.00%

Secretion
---------
Metabolite Reaction       Flux  C-Number   C-Flux
 s_0458[e]   r_1672     -2.097         1  100.00%
 s_0776[e]   r_1814 -4.351E-06       

The growth decreases to 0.067 when the growth media insted of having glucose just have xylose. Now it can also be seen how the  the Xylose transport, the XR and the XDH are active, however there is not Xylitol transport. 